## Test the build and running of containers

# sm-base

In [383]:
!docker build -f ../Dockerfile.base -t sm-base ..

Sending build context to Docker daemon  417.8kB
Step 1/12 : FROM python:3.7
 ---> 8e3336637d81
Step 2/12 : ENV PYTHONDONTWRITEBYTECODE=1 PYTHONUNBUFFERED=1 PYTHONIOENCODING=UTF-8 LANG=C.UTF-8 LC_ALL=C.UTF-8
 ---> Using cache
 ---> cda7dc370c61
Step 3/12 : ENV PATH="/opt/program:${PATH}"
 ---> Using cache
 ---> 1bab301534e5
Step 4/12 : RUN apt update # 17.4MB
 ---> Using cache
 ---> 99d3220a9dac
Step 5/12 : RUN pip install flask # 26.8MB
 ---> Using cache
 ---> 0076d0b6a6e5
Step 6/12 : RUN pip install boto3 # 84.2MB
 ---> Using cache
 ---> d3544692a54a
Step 7/12 : RUN pip install pandas scikit-learn # 392MB
 ---> Using cache
 ---> 69ac9f4f8895
Step 8/12 : RUN rm -rf /root/.cache
 ---> Using cache
 ---> e44fa9a2385b
Step 9/12 : ENV FLASK_ENV=production FLASK_APP=/opt/program/wsgi FLASK_RUN_HOST=0.0.0.0 FLASK_RUN_PORT=8080
 ---> Using cache
 ---> 0678e52fb8b0
Step 10/12 : COPY program /opt/program/
 ---> 4bc5f3aa41f3
Step 11/12 : WORKDIR /opt/program
 ---> Running in 8f0d5c58cf60
Removing

In [275]:
%%bash
docker run --rm sm-base python --version
docker run --rm sm-base python -c "import numpy, pandas, scipy, sklearn; print(numpy.__version__, pandas.__version__, scipy.__version__, sklearn.__version__)"

Python 3.7.7
1.18.2 1.0.3 1.4.1 0.22.2.post1


## Helper script for testing

/tmp/test-build.sh

In [102]:
# %%writefile docker-compose.yml
# version: '2.0'
# services:
#   web:
#     image: sm-base
#     ports:
#     - "8080:8080"
#     volumes:
#     - /dev/shm/userpackage:/opt/program/userpackage
#     environment:
#     - SAGEMAKER_PROGRAM=entry.py
#     - PRINT_SYS=1
#     entrypoint: serve

# !docker-compose 

In [426]:
%%writefile /tmp/test-build.sh
#!/bin/bash
container=$1
data=$2
s=${3:-'6'} # sleep time before issuing http request
SAGEMAKER_PROGRAM=${SAGEMAKER_PROGRAM:-"entry.py"}
PRINT_SYS=${PRINT_SYS:-"1"}

if [[ $PRINT_SYS == '-1' ]]; then PRINT_SYS=''; fi


function init_serve() {
    docker run --rm \
      -v /dev/shm/userpackage:/opt/program/userpackage \
      -p 8080:8080 \
      -e SAGEMAKER_PROGRAM=$SAGEMAKER_PROGRAM \
      -e PRINT_SYS=$PRINT_SYS \
      --name test \
    $container serve
}

# make the ping and invocation request
function req() {
    sleep $s # need to sleep a bit for the server to start up
    echo ping
    curl localhost:8080/ping
    echo invocations
    curl --data "$data" localhost:8080/invocations
}

# finally kill the container
function k() {
    sleep $((s + 1))
#     docker rm -f test
    docker rm -f $(docker ps -aq)
}

req & k & init_serve

Overwriting /tmp/test-build.sh


### Some data for testing

In [260]:
%%bash
d='/dev/shm/userpackage/'
rm -rf $d
mkdir -p $d"insect/hymenoptera/ant"
mkdir -p $d"insect/hymenoptera/bee"
mkdir -p $d"insect/lepidoptera/butterfly"
mkdir -p $d"crustacean/nephropoidea/lobster"

cd $d
echo -e "print('Phylum arthropod')" > arthropod.py
echo -e "print('insect init')" > insect/__init__.py
echo -e "print('red ant'); from . import black" > insect/hymenoptera/ant/red.py
echo -e "print('black ant'); from ..bee import bumble" > insect/hymenoptera/ant/black.py
echo -e "print('bumble bee'); from ...lepidoptera.butterfly import monarch" > insect/hymenoptera/bee/bumble.py
echo -e "print('monarch butterfly');
import os;
path = os.path.dirname(__file__) + '/monarch.txt'
print(open(path).read())

path = os.path.dirname(__file__)
path = os.path.join(path, '../../..', 'crustacean/nephropoidea/lobster/lobster.txt')
print(open(path).read())"> insect/lepidoptera/butterfly/monarch.py

echo -e "monarchs migrate" > insect/lepidoptera/butterfly/monarch.txt
echo -e "lobsters scavenge" > crustacean/nephropoidea/lobster/lobster.txt

In [261]:
%%writefile /dev/shm/userpackage/entry.py
import os
import numpy
import pandas
import scipy
import sklearn

print(f'''
User entry point
numpy: {numpy.__version__}
pandas: {pandas.__version__}
scipy: {scipy.__version__}
{__file__}
dirname: {os.path.dirname(__file__)}
os.getcwd: {os.getcwd()}
''')

if __name__== '__main__':
    print('Main')

def model_fn(model_dir):
    print('model_fn called', model_dir)
    
def transform_fn(input_data, model):
    print('transform_fn called', input_data, model)
    return input_data

from . import arthropod
from .insect.hymenoptera.ant import red

Writing /dev/shm/userpackage/entry.py


In [321]:
# can run the entry.py as a package locally 
!cd /dev/shm && python -c 'import userpackage.entry'


User entry point
numpy: 1.14.3
pandas: 0.24.2
scipy: 1.1.0
/dev/shm/userpackage/entry.py
dirname: /dev/shm/userpackage
os.getcwd: /dev/shm

Phylum arthropod
insect init
red ant
black ant
bumble bee
monarch butterfly
monarchs migrate

lobsters scavenge



In [428]:
!bash /tmp/test-build.sh sm-base "blah blah $RANDOM"

Start SERVE
environ({'PATH': '/opt/program:/usr/local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin', 'HOSTNAME': 'eb02ece9a1e3', 'SAGEMAKER_PROGRAM': 'entry.py', 'PRINT_SYS': '1', 'LANG': 'C.UTF-8', 'GPG_KEY': '0D96DF4D4110E5C43FBFB17F2D347EA6AA65421D', 'PYTHON_VERSION': '3.7.7', 'PYTHON_PIP_VERSION': '20.0.2', 'PYTHON_GET_PIP_URL': 'https://github.com/pypa/get-pip/raw/d59197a3c169cef378a22428a3fa99d33e080a5d/get-pip.py', 'PYTHON_GET_PIP_SHA256': '421ac1d44c0cf9730a088e337867d974b91bdce4ea2636099275071878cc189e', 'PYTHONDONTWRITEBYTECODE': '1', 'PYTHONUNBUFFERED': '1', 'PYTHONIOENCODING': 'UTF-8', 'LC_ALL': 'C.UTF-8', 'FLASK_ENV': 'production', 'FLASK_APP': '/opt/program/wsgi', 'FLASK_RUN_HOST': '0.0.0.0', 'FLASK_RUN_PORT': '8080', 'HOME': '/root'})
pip freeze
boto3==1.12.26
botocore==1.15.26
click==7.1.1
docutils==0.15.2
Flask==1.1.1
itsdangerous==1.1.0
Jinja2==2.11.1
jmespath==0.9.5
joblib==0.14.1
MarkupSafe==1.1.1
numpy==1.18.2
pandas==1.0.3
python-dateutil==2.8.

# sm-xgb

In [276]:
!docker build -f ../Dockerfile.xgb -t sm-xgb ..

Sending build context to Docker daemon  376.8kB
Step 1/3 : FROM sm-base
 ---> acb099b9a6e0
Step 2/3 : RUN pip install xgboost
 ---> Running in bf7043003eac
Removing intermediate container bf7043003eac
 ---> e0074bf22128
Step 3/3 : RUN rm -rf /root/.cache
 ---> Running in 2913ecb692e4
Removing intermediate container 2913ecb692e4
 ---> bef5825a5c61
Successfully built bef5825a5c61
Successfully tagged sm-xgb:latest


In [277]:
!docker run --rm sm-xgb python -c "import xgboost; print(xgboost.__version__)"

1.0.2


# sm-tf

In [384]:
!docker build -f ../Dockerfile.tf -t sm-tf ..

Sending build context to Docker daemon  417.8kB
Step 1/3 : FROM sm-base
 ---> 0ab16925eb2c
Step 2/3 : RUN pip install tensorflow
 ---> Running in e9e903822997
  Created wheel for gast: filename=gast-0.2.2-py3-none-any.whl size=7539 sha256=bfad57d9b20fb78823b1783bc3de4cd569076189a22171c53a9375ce16a11b3c
  Stored in directory: /root/.cache/pip/wheels/21/7f/02/420f32a803f7d0967b48dd823da3f558c5166991bfd204eef3
  Created wheel for termcolor: filename=termcolor-1.1.0-py3-none-any.whl size=4830 sha256=345b3ee4764f59f3db24cc569c0ab7a488991e4d58bb65a85f68f62f25fa4319
  Stored in directory: /root/.cache/pip/wheels/3f/e3/ec/8a8336ff196023622fbcb36de0c5a5c218cbb24111d1d4c7f2
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=76427 sha256=5f5f6d8e8124ccb1655108c96a550399c8bf8c7bba6dbdcd4bbdae2317e752ec
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
  Created wheel for absl-py: filename=absl_py-0.9.0-py3-none


**Can run sm-tf. Ignore the warning messages about CPU, GPU**

In [386]:
%%bash
docker run --rm sm-tf python -c "
import tensorflow as tf;
print(tf.__version__);
print('#####', tf.reduce_sum(tf.random.normal([1000, 1000])))
"

2.1.0
##### tf.Tensor(-609.01776, shape=(), dtype=float32)


2020-03-23 19:20:22.789262: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-23 19:20:22.789424: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-23 19:20:22.789443: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
2020-03-23 19:20:24.389861: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2020-03-23 19:20:24.389900: E tensorflow/stream_executor/cuda

In [420]:
%%writefile /dev/shm/userpackage/entry_tf.py
import os
import numpy
import pandas
import scipy
import sklearn
import tensorflow

print(f'''
User entry point
numpy: {numpy.__version__}
pandas: {pandas.__version__}
scipy: {scipy.__version__}
sklearn: {sklearn.__version__}
tf: {tensorflow.__version__}
{__file__}
dirname: {os.path.dirname(__file__)}
os.getcwd: {os.getcwd()}
''')


#### Some testing ###

import numpy as np
import tensorflow as tf
import json
import subprocess

n = 6000 # rows

np.random.seed(0)

X = np.random.rand(n, 2) * np.random.choice([-1, 1], (n, 2))
y = 3 + 10 * X[:,0] + 20 * X[:,1] + np.random.rand(n) * np.random.choice([-1, 1], n)

# split into train and test
split = 5000
X_train, X_val = X[:split,], X[split:,]
y_train, y_val = y[:split,], y[split:,]

model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(4, input_dim=2, activation='sigmoid'))
model.add(tf.keras.layers.Dense(2, activation='linear'))
model.add(tf.keras.layers.Dense(1))

model.compile(optimizer='rmsprop', loss='mean_squared_error', metrics=['mae'])

model.fit(X_train, y_train, epochs=1, verbose=2)

print('\n###### Training completed #####')
subprocess.run(['mkdir', '-p', '/opt/ml/model'])
model.save('/opt/ml/model/model.h5')
print('\n###### Model saved #####')

print(((model.predict(X_val).flatten() - y_val) ** 2).mean() ** 0.5)

###### the inference part ##########

if __name__== '__main__':
    print('Main')

def model_fn(model_dir):
    print('model_fn called', model_dir)
    m = tf.keras.models.load_model(model_dir + 'model.h5')
    print('Model loaded')
    return m
    
    
def transform_fn(input_data, model):
    print('transform_fn called', input_data, model)
    tmp = json.loads(input_data)
    tmp = np.array(tmp)
    tmp = model.predict(tmp)
    print('the predicted', tmp)
    return np.array2string(tmp)

Overwriting /dev/shm/userpackage/entry_tf.py


In [427]:
!PRINT_SYS="-1" SAGEMAKER_PROGRAM="entry_tf.py" bash /tmp/test-build.sh sm-tf "[[1, 2], [3, 4]]" 15

Start SERVE
2020-03-23 19:37:04.342680: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2020-03-23 19:37:04.342996: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2020-03-23 19:37:04.343022: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.

User entry point
numpy: 1.18.2
pandas: 1.0.3
scipy: 1.4.1
sklearn: 0.22.2.post1
tf: 2.1.0
/opt/program/userpackage/entry_tf.py
dirname: /opt/program/userpackage
os.getcwd: /opt/program

2020-03-23 19:37:04.979118: W tensorflow/stream_executor/platform/default/dso_load